<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"
var actual_EstimateTypes = new[] {"A","AA","OA","DA"};
var args_Best_Estimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var args_Test_Scenario = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "Test", ImportFormats.Cashflow);

In [0]:
public async Task<ActivityLog> ImportAndInitializeWorkspaceAsync(string inputString, string importFormat, bool fromFile = true)
{   
    var log = fromFile 
                    ? await Import.FromFile(inputString).WithFormat(importFormat).WithTarget(DataSource).ExecuteAsync()
                    : await Import.FromString(inputString).WithFormat(importFormat).WithTarget(DataSource).ExecuteAsync();
    Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    Workspace.Initialize(x => x.FromSource(DataSource));
    return log;
}

# Context

In [0]:
await ImportAndInitializeWorkspaceAsync("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv", ImportFormats.Cashflow)

In [0]:
await ImportAndInitializeWorkspaceAsync("../Files/TransactionalData/Actuals_CH_2020_12.csv", ImportFormats.Actual)

In [0]:
var defaultVarsBE = Workspace.Query<IfrsVariable>().ToArray();

# Scenario Cash flows

In [0]:
var cashflows_scenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var cashflows_scenarioTest_equalTo_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";

In [0]:
await ImportAndInitializeWorkspaceAsync(cashflows_scenarioTest, ImportFormats.Cashflow, false)

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().Be(0);
diffs.Where(x => !actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

# Scenario Actuals

In [0]:
var actuals_scenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-308";

In [0]:
var actuals_scenarioTest_equalTo_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-280";

In [0]:
await ImportAndInitializeWorkspaceAsync(actuals_scenarioTest, ImportFormats.Actual, false)

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

# Robustness: back to Best Estimate

In [0]:
await ImportAndInitializeWorkspaceAsync(actuals_scenarioTest_equalTo_BestEstimate, ImportFormats.Actual, false)

In [0]:
await ImportAndInitializeWorkspaceAsync(cashflows_scenarioTest_equalTo_BestEstimate, ImportFormats.Cashflow, false)

In [0]:
Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);

# Reimport of Best Estimate Cashflow

In [0]:
var new_Cashflows_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,CU,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42";

In [0]:
await ImportAndInitializeWorkspaceAsync(new_Cashflows_BestEstimate, ImportFormats.Cashflow, false)

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Best_Estimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Best_Estimate);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);
diffs.Where(x => !actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);
diffs.Where(x => !actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

# Reimport of Best Estimate Actuals

In [0]:
var new_actuals_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-2000";

In [0]:
await ImportAndInitializeWorkspaceAsync(new_actuals_BestEstimate, ImportFormats.Actual, false)

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Best_Estimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Best_Estimate);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args_Test_Scenario);
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => actual_EstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);